In [1]:
import spacy
import numpy as np
import re

nlp = spacy.load('en_core_web_sm')

In [2]:
def nGrams(doc, n, isTokenized):
    if not isTokenized:
        doc = [token.text for token in nlp(doc)]

    doc = ' '.join(doc).lower().split(' ') #convert all to lowercase
    grams = [doc[i:i+n] for i in range(len(doc)-n+1)]
    # print (doc)
    return grams

In [3]:
doc = 'How old are you today or can you tell me something about yourself'
# doc = ['How', 'old', 'are', 'you', 'today']
n = 4
grams = nGrams(doc, n, False)
grams

[['how', 'old', 'are', 'you'],
 ['old', 'are', 'you', 'today'],
 ['are', 'you', 'today', 'or'],
 ['you', 'today', 'or', 'can'],
 ['today', 'or', 'can', 'you'],
 ['or', 'can', 'you', 'tell'],
 ['can', 'you', 'tell', 'me'],
 ['you', 'tell', 'me', 'something'],
 ['tell', 'me', 'something', 'about'],
 ['me', 'something', 'about', 'yourself']]

In [4]:
from collections import defaultdict

In [5]:
def buildModel():
    model = defaultdict(lambda: defaultdict(lambda: 0)) #eg. {x: {y: 0}}
    return model

In [6]:
def updateCount(nGram, model):
    w_1_to_n_minus_1 = tuple(nGram[:-1])
    w_n = nGram[-1]
    model[w_1_to_n_minus_1][w_n] += 1 #eg. {w_1_to_n_minus_1: {w_n: 1}}
    return model

In [7]:
def computeProbability(model):
    for w_1_to_n_minus_1 in model:
        totalCount = float(sum(model[w_1_to_n_minus_1].values()))
        for w_n in model[w_1_to_n_minus_1]:
            model[w_1_to_n_minus_1][w_n] /= totalCount
    return model

In [8]:
# !pip install dill #extention of pickle
import dill

In [9]:
def saveModel(model, fileName):
    with open('../Models/'+fileName, 'wb') as f:
        dill.dump(model, f)

def loadModel(fileName):
    with open('../Models/'+fileName, 'rb') as f:
        model = dill.load(f)
    return model

In [10]:
# !pip install nltk #For the 1st time only
import nltk

In [11]:
from sklearn.datasets import fetch_20newsgroups as getData
# nltk.download('reuters') #For the 1st time only
from nltk.corpus import reuters as corpus

In [12]:
getData

<function sklearn.datasets._twenty_newsgroups.fetch_20newsgroups(*, data_home=None, subset='train', categories=None, shuffle=True, random_state=42, remove=(), download_if_missing=True, return_X_y=False, n_retries=3, delay=1.0)>

In [13]:
# X,y = getData(subset='train',remove=('headers','footers','qoutes'),return_X_y=True) #didn'swork directly like the Udemy

In [14]:
# import ssl
# import urllib.request

# ssl._create_default_https_context = ssl._create_unverified_context
# opener = urllib.request.build_opener(urllib.request.HTTPSHandler(context=ssl._create_unverified_context()))
# urllib.request.install_opener(opener)

# from sklearn.datasets import fetch_20newsgroups
X, y = getData(subset='train', remove=('headers', 'footers', 'quotes'), return_X_y=True)


In [15]:
print(X[0])
print(y[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.
7


In [16]:
len(X)

11314

In [17]:
n = 3
model = buildModel()

In [18]:
for doc in X:
    for nGram in nGrams(doc, n, False):
        model = updateCount(nGram, model)

In [19]:
# nltk.download('punkt_tab') #For the 1st time only

for sentence in corpus.sents():
    for nGram in nGrams(sentence, n, True):  #Data is tokenized here. So True
        model = updateCount(nGram, model)

In [20]:
model = computeProbability(model)

In [21]:
saveModel(model, 'nGram_model.pkl')

In [105]:
text = ['after','that']
nextWords = list(model[tuple(text[-n+1:])].keys()) #-n+ means last n-1 words from given list
probs = list(model[tuple(text[-n+1:])].values())
nextWord = np.random.choice(nextWords,1)[0]
print(nextWord)

thou


In [162]:
def sampleText(model, startingText=['after','that'], maxLength=100, nGramSize=3):
    text = startingText
    n = nGramSize
    while not len(text)>maxLength:
        nextWords = list(model[tuple(text[-n+1:])].keys()) #-n+ means last n-1 words from given list
        probs = list(model[tuple(text[-n+1:])].values())
        # if len(nextWords) > 0:
        #     nextWord = np.random.choice(nextWords,1,probs)[0]
        #     text.append(nextWord)
        # else:
        #     break
            
        if len(nextWords) > 0:
            nextWord = nextWords[np.argmax(probs)]
            text.append(nextWord)
        else:
            break
        
    sampled = ' '.join(text)
    return sampled

In [164]:
for s in nlp(sampleText(model, ['what','if'])).sents:
    print(s)
    break

what if they care 
 : kirillian photography interact with mime is a hot dog he was considering taking the required location is static . 
 
